# Usage example

This notebook aims at providing some example code on how to use the python package CarbonAImpact.
This package allows you to monitor the energy consumption and CO2 footprint of any sample of code or function

[TOC]

In all this notebook we'll try to measure how much CO2 does the training of a toy algorithm on the mnist database emit
```python
mnist = datasets.load_digits()
X = mnist.data
y = mnist.target
clf = SGDClassifier(loss='log', alpha=1e-5)
cv = StratifiedKFold(10, random_state=0, shuffle=True)
cv_results = cross_validate(clf, X, y, cv=cv)
print(cv_results['test_score'].mean(), cv_results['test_score'].std())
```

In [1]:
from CarbonAImpact import PowerMeter

In [2]:
help(PowerMeter)

Help on class PowerMeter in module CarbonAImpact.PowerMeter:

class PowerMeter(builtins.object)
 |  PowerMeter(project_name='', program_name='', client_name='', cpu_power_log_path='', get_country=True, user_name='', filepath=None, api_endpoint=None, location='', is_online=True, output_format='csv')
 |  
 |  PowerMeter is a general tool to monitor and log the power consumption of any given function.
 |  
 |  Parameters
 |  ----------
 |  - project_name (optional) : str
 |      Name of the project you are working on (default is folder_name)
 |  - cpu_power_log_path (optional) : str
 |      The path to the tool "PowerLog"
 |  - get_country (optional) : bool
 |      Whether use user country location or not
 |  - user_name (optional) : str
 |      The name of the user using the tool (for logging purpose)
 |  - filepath (optional) : str
 |      Path of the file where all the green ai logs are written
 |  - api_endpoint (optional):
 |      Endpoint of the API
 |  
 |  Methods defined here:
 |

In [3]:
# imports
from CarbonAImpact import PowerMeter #Our package

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import SGDClassifier

In [9]:
# Creates a power meter object that contains information relative to the current project
# You need to do this step no matter how you use the package
power_meter = PowerMeter(project_name="Usage examples", is_online=False, location="FR")

## Function decorator

In [10]:
# Add a decorator to the main function to measure power usage of this function each time it is called
@power_meter.measure_power(
  package="sklearn",
  algorithm="SGDClassifier",
  data_type="tabular/images",
  data_shape="(1797, 64)",
  algorithm_params="loss='log', alpha=1e-5",
  comments="10 fold cross validated training of logistic regression classifier trained on the MNIST dataset"
)
def cross_val_mnist(alpha, random_state=0):
    mnist = datasets.load_digits()
    X = mnist.data
    y = mnist.target
    clf = SGDClassifier(loss='log', alpha=alpha, random_state=random_state)
    cv = StratifiedKFold(10, random_state=random_state, shuffle=True)
    cv_results = cross_validate(clf, X, y, cv=cv)
    print(cv_results['test_score'].mean(), cv_results['test_score'].std())
    return cv_results

In [11]:
train_results = cross_val_mnist(1e-5)

0.9565983860955928 0.013556136197153262


## Magic function
Only usable in notebooks

In [12]:
# import the magic function module
%load_ext CarbonAImpact.MagicPowerMeter

In [13]:
%%measure_power?

Docstring:
::

  %measure_power [--data_type DATA_TYPE] [--data_shape DATA_SHAPE]
                     [--algorithm_params ALGORITHM_PARAMS]
                     [--comments COMMENTS]
                     power_meter package algorithm

positional arguments:
  power_meter           The PowerMeter object of this project
  package               The name of the package used here
  algorithm             The algorithm type used here

optional arguments:
  --data_type DATA_TYPE
                        Type of data used
  --data_shape DATA_SHAPE
                        Size of the data used
  --algorithm_params ALGORITHM_PARAMS
                        Some informative parameters used in your algorithm
  --comments COMMENTS   Comments to describe what is done
File:      /usr/local/anaconda3/envs/green_ai/lib/python3.7/site-packages/CarbonAImpact/MagicPowerMeter.py


In [14]:
%%measure_power power_meter "sklearn" "SGDClassifier" --data_type "tabular/images" --data_shape "(1797,64)" --algorithm_params "loss='log', alpha=1e-5" --comments "10 fold cross validated training of logistic regression classifier trained on the MNIST dataset"
# This command must always be on the first line of the cell and must be written on one line
mnist = datasets.load_digits()
X = mnist.data
y = mnist.target
clf = SGDClassifier(loss='log', alpha=1e-5, random_state=0)
cv = StratifiedKFold(10, random_state=0, shuffle=True)
cv_results = cross_validate(clf, X, y, cv=cv)
print(cv_results['test_score'].mean(), cv_results['test_score'].std())

0.9565983860955928 0.013556136197153262


## Inline commands
There are 2 ways of using this package using inline commands, we recommmand using the *with statement* over the *start and stop methods* as the with statement stops the process automatically even if you get an error in the process

### Using the with statement

In [15]:
with power_meter(
  package="sklearn",
  algorithm="SGDClassifier",
  data_type="tabular/images",
  data_shape="(1797, 64)",
  algorithm_params="loss='log', alpha=1e-5",
  comments="10 fold cross validated training of logistic regression classifier trained on the MNIST dataset"
):
    mnist = datasets.load_digits()
    X = mnist.data
    y = mnist.target
    clf = SGDClassifier(loss='log', alpha=1e-5, random_state=0)
    cv = StratifiedKFold(10, random_state=0, shuffle=True)
    cv_results = cross_validate(clf, X, y, cv=cv)
    print(cv_results['test_score'].mean(), cv_results['test_score'].std())

0.9565983860955928 0.013556136197153262


### Using start_measure and stop_measure methods

In [16]:
power_meter.start_measure(
  package="sklearn",
  algorithm="SGDClassifier",
  data_type="tabular/images",
  data_shape="(1797, 64)",
  algorithm_params="loss='log', alpha=1e-5",
  comments="10 fold cross validated training of logistic regression classifier trained on the MNIST dataset"
)

mnist = datasets.load_digits()
X = mnist.data
y = mnist.target
clf = SGDClassifier(loss='log', alpha=1e-5, random_state=0)
cv = StratifiedKFold(10, random_state=0, shuffle=True)
cv_results = cross_validate(clf, X, y, cv=cv)
print(cv_results['test_score'].mean(), cv_results['test_score'].std())

power_meter.stop_measure()

0.9565983860955928 0.013556136197153262
